## 1. Read the dataset

In [22]:
import pandas as pd

train_data_original = pd.read_csv("basket_data_by_date_train.csv")
test_data_original = pd.read_csv("basket_data_by_date_test.csv")

train_data_big = pd.read_csv("basket_data_by_date_train_big.csv")
test_data_big = pd.read_csv("basket_data_by_date_test_big.csv")

Check the structure and content of the dataset

In [23]:
print("Train Data Info:")
print(train_data_original.info())

print("\nTest Data Info:")
print(test_data_original.info())

Train Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BillNo      40000 non-null  int64  
 1   Itemname    40000 non-null  object 
 2   Quantity    40000 non-null  int64  
 3   Date        40000 non-null  object 
 4   Price       40000 non-null  float64
 5   CustomerID  40000 non-null  int64  
 6   cost        40000 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 2.1+ MB
None

Test Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BillNo      10000 non-null  float64
 1   Itemname    10000 non-null  object 
 2   Quantity    10000 non-null  float64
 3   Date        10000 non-null  object 
 4   Price       10000 non-null  float64
 5   CustomerID  10000 non-

In [16]:
print("Train Data big Info:")
print(train_data_big.info())

print("\nTest Data big Info:")
print(test_data_big.info())

Train Data big Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      200000 non-null  int64  
 1   Itemname    200000 non-null  object 
 2   Quantity    200000 non-null  int64  
 3   Date        200000 non-null  object 
 4   Price       200000 non-null  float64
 5   CustomerID  200000 non-null  int64  
 6   cost        200000 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 10.7+ MB
None

Test Data big Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65326 entries, 0 to 65325
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BillNo      65326 non-null  int64  
 1   Itemname    65326 non-null  object 
 2   Quantity    65326 non-null  int64  
 3   Date        65326 non-null  object 
 4   Price       65326 non-null  float64
 5   Cu


Based on the information provided about the data, it can be seen that both the training and test sets contain seven columns, which are:

BillNo, Itemname, Quantity, Date, Price, CustomerID, cost

## 2. Handling of missing values

In [19]:
# check if there are missing values in the train set
print("Train Data Missing Values:")
print(train_data_original.isnull().sum())

# check if there are missing values in the test set
print("\nTest Data Missing Values:")
print(test_data_original.isnull().sum())

Train Data Missing Values:
BillNo        0
Itemname      0
Quantity      0
Date          0
Price         0
CustomerID    0
cost          0
dtype: int64

Test Data Missing Values:
BillNo        30000
Itemname      30000
Quantity      30000
Date          30000
Price         30000
CustomerID    30000
cost          30000
dtype: int64


In [20]:
# check if there are missing values in the train set(big)
print("Train Data Big Missing Values:")
print(train_data_big.isnull().sum())

# check if there are missing values in the test set(big)
print("\nTest Data Big Missing Values:")
print(test_data_big.isnull().sum())

Train Data Big Missing Values:
BillNo        0
Itemname      0
Quantity      0
Date          0
Price         0
CustomerID    0
cost          0
dtype: int64

Test Data Big Missing Values:
BillNo        0
Itemname      0
Quantity      0
Date          0
Price         0
CustomerID    0
cost          0
dtype: int64


The data shows that the dataset of Test Data contains a large number of missing values. So I will deal with these missing values.